In [1]:
import requests
import json
import pandas as pd

In [2]:
url = 'https://api.mainnet-beta.solana.com'
headers = {'content-type': 'application/json',}

In [12]:
def get_vote_accounts(url, headers):
    '''Getting all vote accounts'''
    payload = json.dumps({
        'jsonrpc': '2.0',
        'id': 1,
        'method': 'getVoteAccounts',
        'params': [
        ]
    })
    
    response = requests.post(url, data=payload, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Request failed with status code {response.status_code}")

In [34]:
def filter_accounts_by_epoch(response, epoch):
    '''Filtering for Vote Accounts active in 598'''
    active_during_epoch = []
    for validator in response['result']['current']:
        for epoch_credit in validator['epochCredits']:
            if epoch_credit[0] == epoch:
                active_during_epoch.append({
                    'VotePubkey': validator['votePubkey'],
                    'ActivatedStake': round(validator['activatedStake']/1e9,2), #converting to sol
                    'Commission': validator['commission'],
                    'VoteCredits': epoch_credit[1] - epoch_credit[2]
                })
                break
    return active_during_epoch

In [35]:
try:
    res = get_vote_accounts(url, headers)
    vote_accounts_598 = filter_accounts_by_epoch(res, 598)
except Exception as e:
    print(e)

In [56]:
df_accounts = pd.DataFrame(vote_accounts_598)
df_accounts


,VotePubkey,ActivatedStake,Commission,VoteCredits
0,42U2eB71MKU7mqVpy5b63meM7Pd4YBqwAGSDevjJ9E9z,20251.99,10,360398
1,GZCv21mPm7HNwiC5Hq3j1DXz4njDQownDAfN7xziXbjN,18126.14,0,360807
2,4TPBarwgZuuztwD7biHtt87L2BTmH4KjcePABijDeiyW,8698.25,100,360506
3,F4UVoVGgmd2aJwnfz3eojQ92P9AvYbRG51z1xnEj8tFy,36409.71,0,361022
4,Ev683W2Xe56cJDG9Ja5rZDSVgWk1e5NVug3iJUWP2VZL,10059.07,100,360659
...,...,...,...,...
1723,8EQwZusrHgjZyDKXVdtsCeGNsL6VmEB5xq2EP6XK4AUq,36660.61,7,360650
1724,GgdUMBPn947xxHxc9ytQywjKeQcVD9AWR3EspNEc3iBi,37216.57,0,360831
1725,EWQfCyftKLVbeJtJ4idbLDcDFSGaZ6bZHGS5vboUx3VR,52477.10,7,360535
1726,5afRnmkFn1pRU9oussqwk1RRBVyoDgUkL16Jz4qNf574,48892.71,7,360118


In [76]:
def get_rewards_by_epoch(vote_pubkeys, epoch, url, headers):
    '''Getting the inflation rewards for each validator in a given epoch'''
    rewards = {}
    batch_size = len(vote_pubkeys) // 2

    for i in range(0, len(vote_pubkeys), batch_size):
        batch = vote_pubkeys[i:i + batch_size]
        payload = json.dumps({
            'jsonrpc': '2.0',
            'id': 1,
            'method': 'getInflationReward',
            'params': [
                batch, 
                {'epoch': epoch}
            ]
        })

        response = requests.post(url, data=payload, headers=headers)
        if response.status_code == 200:
            data = response.json()
            if 'result' in data and data['result'] is not None:
                for pubkey, result in zip(batch, data['result']):
                    rewards[pubkey] = result
        else:
            print(f"Failed, starting at index {i}: Status code {response.status_code}")

    return rewards


In [68]:
try:
    rewards_by_epoch = get_rewards_by_epoch(df_accounts['VotePubkey'].to_list(), 598, url, headers)
except Exception as e:
    print("An error occurred:", e)

In [69]:
rows = []
for pubkey, details in rewards_by_epoch .items():
    row = {
        'VotePubkey': pubkey,
        'Reward': round(details['amount']/1e9,2), #Converting to SOL
    }
    rows.append(row)

df_rewards = pd.DataFrame(rows)
df_rewards

In [71]:
df_merged = pd.merge(df_rewards, df_accounts, on='VotePubkey', how='outer')
df_merged = df_merged[['VotePubkey','VoteCredits','ActivatedStake','Commission','Reward']]
df_merged

In [75]:
df_merged.to_csv('VoteRewards598.csv', index = False)